In [2]:
from tokenizers import (
    decoders,
    models,
    pre_tokenizers,
    trainers,
    normalizers,
    Tokenizer,
)
from transformers import PreTrainedTokenizerFast
from pathlib import Path

In [3]:
data_paths = [
    "/media/luischary/BIGGER/DATASETS/NLP/AlpacaPT/textos",
    "/media/luischary/BIGGER/DATASETS/NLP/Livros/textos_extraidos",
    "/media/luischary/BIGGER/DATASETS/NLP/OUTROS/corpus_readability_nlp_portuguese-master/1_Ensino_Fundamental_I",
    "/media/luischary/BIGGER/DATASETS/NLP/OUTROS/corpus_readability_nlp_portuguese-master/2_Ensino_Fundamental_II",
    "/media/luischary/BIGGER/DATASETS/NLP/OUTROS/corpus_readability_nlp_portuguese-master/3_Ensino_Medio",
    "/media/luischary/BIGGER/DATASETS/NLP/OUTROS/corpus_readability_nlp_portuguese-master/4_Ensino_Superior",
    "/media/luischary/BIGGER/DATASETS/NLP/OUTROS/enem_2022/textos",
    "/media/luischary/BIGGER/DATASETS/NLP/Summarization/portuguese_XLSum_v2/textos_train",
    "/media/luischary/BIGGER/DATASETS/NLP/Summarization/TeMario-ULTIMA VERSAO out2004/textos",
    "/media/luischary/BIGGER/DATASETS/NLP/Traducao/textos_capes",
]


def iterate_over_texts():
    for datafolder in data_paths:
        input_path = Path(datafolder)
        for arquivo_de_texto in input_path.glob("*.txt"):
            yield arquivo_de_texto.read_text(encoding="utf8")

In [4]:
frase_teste = "Esta é a minha primeira frase maçante. Senhor cuzão 12345"

In [13]:
# cria o tokenizer
tokenizer = Tokenizer(models.BPE())

# cria o pre-tokenizer
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
# essa opcao de add_prefix_space serve para adicionar um espaco no comeco.
# como eh falsa, nao vamos adicionar um espaco no comeco da string.

# teste do pre-tokenizer
print(tokenizer.pre_tokenizer.pre_tokenize_str(frase_teste))

# bota um normalizer para substituir algumas coisas
tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("\n(.)\n", "\n"),
        normalizers.Replace("[\n\n]+", "\n\n"),
        # normalizers.NFC(),
        # normalizers.NFD(),
        # normalizers.NFKC(),
        normalizers.NFKD(),
    ]
)

# cria o trainer do tokenizer
trainer = trainers.BpeTrainer(
    vocab_size=50000,
    special_tokens=[
        "<pad>",
        "<s>",
        "</s>",
        "<mask>",
        "<cls>",
        "<question>",
        "<answer>",
        "<unk>",
    ],
)
tokenizer.train_from_iterator(iterate_over_texts(), trainer=trainer)

# agora, para nao ficar com aquele G que representa o espaco no comeco da palavra quando
# detokenizarmos o texto
# para isso colocarmos um decoder de bytelevel
tokenizer.decoder = decoders.ByteLevel()

# no final temos que salvar ele de alguma forma
# vamos usar a classe basica de PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>",
    cls_token="<cls>",
)

[('Esta', (0, 4)), ('ĠÃ©', (4, 6)), ('Ġa', (6, 8)), ('Ġminha', (8, 14)), ('Ġprimeira', (14, 23)), ('Ġfrase', (23, 29)), ('ĠmaÃ§ante', (29, 37)), ('.', (37, 38)), ('ĠSenhor', (38, 45)), ('ĠcuzÃ£o', (45, 51)), ('Ġ12345', (51, 57))]





In [14]:
#NFKD
tokenizado = wrapped_tokenizer.encode(frase_teste)
detokenizado = wrapped_tokenizer.decode(tokenizado)
print(frase_teste)
print(len(tokenizado))
print(detokenizado)

Esta é a minha primeira frase maçante. Senhor cuzão 12345
19
Esta é a minha primeira frase maçante. Senhor cuzão 12345


In [12]:
#NFKC
tokenizado = wrapped_tokenizer.encode(frase_teste)
detokenizado = wrapped_tokenizer.decode(tokenizado)
print(frase_teste)
print(len(tokenizado))
print(detokenizado)

Esta é a minha primeira frase maçante. Senhor cuzão 12345
16
Esta é a minha primeira frase maçante. Senhor cuzão 12345


In [10]:
#NFD
tokenizado = wrapped_tokenizer.encode(frase_teste)
detokenizado = wrapped_tokenizer.decode(tokenizado)
print(frase_teste)
print(len(tokenizado))
print(detokenizado)

Esta é a minha primeira frase maçante. Senhor cuzão 12345
19
Esta é a minha primeira frase maçante. Senhor cuzão 12345


In [8]:
#NFC
tokenizado = wrapped_tokenizer.encode(frase_teste)
detokenizado = wrapped_tokenizer.decode(tokenizado)
print(frase_teste)
print(len(tokenizado))
print(detokenizado)

Esta é a minha primeira frase maçante. Senhor cuzão 12345
16
Esta é a minha primeira frase maçante. Senhor cuzão 12345


In [6]:
#normal
tokenizado = wrapped_tokenizer.encode(frase_teste)
detokenizado = wrapped_tokenizer.decode(tokenizado)
print(frase_teste)
print(len(tokenizado))
print(detokenizado)

Esta é a minha primeira frase maçante. Senhor cuzão 12345
16
Esta é a minha primeira frase maçante. Senhor cuzão 12345
